# Perplexity playbook

<p>
Mal Minhas, v0.2<br>
01.01.24
</p>

## 1. Introduction

Perplexity is an intelligent search engine with a chatbot sytle interface that leverages popular AI models to offer comprehensive answers to user inquiries.  It is able to search the internet to provide real-time information via a friendly user experience that includes native mobile apps.  The tool is specifically designed to perform as a research companion providing links to underlying source articles unlike ChatGPT in default mode.  Perplexity comes with a limited API.

Supported models are [as follows](https://docs.perplexity.ai/docs/model-cards):
```
Model	                  Context Length       Model Type
codellama-34b-instruct	      16384          Chat Completion
llama-2-70b-chat               4096          Chat Completion
mistral-7b-instruct [2]	       4096 [1]      Chat Completion
mixtral-8x7b-instruct	       4096 [1]      Chat Completion
pplx-7b-chat	               8192          Chat Completion
pplx-70b-chat	               4096          Chat Completion
pplx-7b-online	               4096          Chat Completion
pplx-70b-online	               4096          Chat Completion

[1] We will be increasing the context length of mistral-7b-instruct to 32k tokens (see roadmap).
[2] This model refers to the v0.2 release of mistral-7b-instruct.
```

The only API function currently supported is on `chat_completions` documented [here](https://docs.perplexity.ai/reference/post_chat_completions).  The next section outlines how to use it.

## 2. Basic Usage

Let's start with a few basics:

In [1]:
import os
import requests

API_KEY = os.environ.get("PERPLEXITY_API_KEY")
DEFAULT_MODEL = "mistral-7b-instruct"
CONCISE_DETAIL = "Be precise and concise."
MID_LEVEL_DETAIL = "Explain in a few sentences."
GREAT_DETAIL = "Explain in a lot of detail."

The following is a convenience function that wraps the Perplexity API allowing different models to be used and also differing levels of depth of response detail:

In [2]:
def getPerplexityResponse(question, model=DEFAULT_MODEL, instruction=MID_LEVEL_DETAIL):
    url = "https://api.perplexity.ai/chat/completions"
    payload = {
        "model": model,
        "messages": [
            {
                "role": "system",
                "content": instruction,
            },
            {
                "role": "user",
                "content": question
            }
        ]
    }
    headers = {
        "accept": "application/json",
        "content-type": "application/json",
        "Authorization": f"Bearer {API_KEY}",
    }
    response = requests.post(url, json=payload, headers=headers).json()
    return response.get("choices")[0].get("message").get("content")

Let's test the function on an example query using the default Mistral 7b model and then Llama2.  Note the differences in response timing:

In [3]:
%%time

getPerplexityResponse("I have a broken washing machine and I live in Southamption in the UK.  What can I do to get it mended?  What trade do I need to call?")

CPU times: user 9.05 ms, sys: 5.64 ms, total: 14.7 ms
Wall time: 1.49 s


"In Southampton, UK, if you have a broken washing machine, you should consider calling a local appliance repair service. These professionals have the expertise to diagnose and fix various washing machine issues. Look up reputable repair companies online, check reviews, and ensure they are certified and insured. Contact them to schedule an appointment, provide them with the model number of your washing machine, and briefly explain the symptoms you've observed. Remember to ask about their estimated cost for the repair before they start the work"

In [7]:
%%time

getPerplexityResponse("I have a broken washing machine and I live in Southamption in the UK.  What can I do to get it mended?  What trade do I need to call?", model='llama-2-70b-chat')

CPU times: user 6.67 ms, sys: 5.52 ms, total: 12.2 ms
Wall time: 2.61 s


"You can call a professional appliance repair technician or a washing machine repair service in Southampton to fix your broken washing machine. You can search for local repair services online or look up reviews on websites like Checkatrade or Trustpilot to find a reputable technician. Alternatively, you can also contact the manufacturer's customer service number to see if they can provide assistance or recommend a repair service."

Let's see how it handles a query with a more detailed response:

In [5]:
%%time

getPerplexityResponse("I have a broken garden gate which is no longer closing properly.  What trade do I need to call to fix it?", instruction=GREAT_DETAIL)

CPU times: user 6.49 ms, sys: 5.72 ms, total: 12.2 ms
Wall time: 4.93 s


"To repair a broken garden gate and ensure it closes properly, you would typically need to hire the services of a tradesperson specializing in carpentry, fence repair, or gate installation and repair. happens, the gate may require various types of repairs ranging from simple adjustments to more complex replacements of broken components. Let me explain the potential issues in detail and what each trade might contribute to fixing them.\n\n1. Carpenter: A carpenter specializes in the designing, constructing, and repairing of wooden structures. If your garden gate is made of wood, a carpenter can assess the problem and perform necessary repairs. They can check for issues like warped wood, broken hinges, or loose screws that might prevent the gate from closing properly. They can also replace damaged wooden parts, tighten screws and nails, or make adjustments to the gate's frame, ensuring a secure and functional gate.\n\n2. Fence Repair Specialist: A fence repair specialist deals specificall